In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
import warnings

warnings.filterwarnings("ignore")
_ = load_dotenv(find_dotenv())

In [3]:
import agentops

agentops.init(api_key=os.environ["AGENTOPS_API_KEY"])

🖇 AgentOps: Only Anthropic>=0.32.0 supported. v0.28.1 found.
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=573a99df-31dd-4ff2-9a96-bd8bfbb29b7e


In [4]:
from autogen import (
    AssistantAgent,
    GroupChat,
    GroupChatManager,
    UserProxyAgent,
)
from autogen.cache import Cache
from autogen.coding import LocalCommandLineCodeExecutor

from datetime import datetime

config_list=[
    {"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}
]

## Defining our task

In [5]:
task = (
    f"Today is {datetime.now().date()}. Write a blogpost about the stock price performance of Illumina in the past month."
)
print(task)

Today is 2024-11-02. Write a blogpost about the stock price performance of Illumina in the past month.


## The Group Chat

In [6]:
## This is us!
user_proxy = UserProxyAgent(
    name="Admin",
    system_message="A human admin. Give the task and send instructions to writer to refine the blog",
    code_execution_config=False
)

## The agents
planner = AssistantAgent(
    name="Planner",
    system_message="""You are a planner. Given a task, please determine what information is needed
    to complete the task. Do note that all the information will be retrieved using Python code, so 
    only give suggestions that can be retrieved by Python code.""",
    llm_config={"config_list": config_list, "cache_seed": None},
)

engineer = AssistantAgent(
    name="Engineer",
    llm_config={"config_list": config_list, "cache_seed": None},
    system_message="""You are a software engineer. You write python/bash to retrieve relevant 
    information. Wrap the code in a code block that specifies the script type. The user cannot
    modify your code so do not suggest incomplete code. Do not use a code block if it's not intended
    to be executed. Don't include multiple code blocks in one response. Do not ask others to copy
    and paste the result. Check the execution result returned by the executor. If the result
    indicates that there is an error, fix the error and output the code again. Suggest the full code
    instead of partial code or code changes. If the error can't be fixed or if the task is not 
    solved even after the code is successfully executed, analyze the problem, revisit your assumptions,
    collec additional info you need, and think of a different approach to try."""
)

writer = AssistantAgent(
    name="Writer",
    llm_config={"config_list": config_list, "cache_seed": None},
    system_message="""You are a writer. Please write blogs in markdown format (with relevant titles)
    and put them in pseudo ```md``` code blocks. You will write it for a task based on previous chat
    history. Don't write any code.""",
)

if not os.path.exists("blogs"):
    os.makedirs("blogs")

code_executor = UserProxyAgent(
    name="Executor",
    system_message="You are a code executor. Execute the code written by the engineer and report the result.",
    description="Executor should always be called after the engineer has written code to be executed.",
    human_input_mode="ALWAYS",
    code_execution_config={
        "last_n_messages": 3,
        "executor": LocalCommandLineCodeExecutor(work_dir="blogs")
    }
)

In [7]:
groupchat = GroupChat(
    agents = [user_proxy, engineer, code_executor, writer, planner],
    messages=[],
    max_round=20,
    speaker_selection_method="auto"
)

manager = GroupChatManager(
    groupchat=groupchat,
    llm_config={"config_list": config_list, "cache_seed": None}
)

In [8]:
## Cache LLM response
with Cache.disk(cache_seed=41) as cache:
    chat_history = user_proxy.initiate_chat(
        manager,
        message=task,
        cache=cache
    )

Admin (to chat_manager):

Today is 2024-11-02. Write a blogpost about the stock price performance of Illumina in the past month.

--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

To write a blog post about the stock price performance of Illumina in the past month, you will need the following information, which can be retrieved using Python code:

1. **Historical Stock Prices**: 
   - Retrieve the stock price data for Illumina (ticker symbol: ILMN) for the past month (from 2024-10-02 to 2024-11-02).

2. **Price Changes**:
   - Calculate the percentage change in the stock price over the month.
   - Identify the highest and lowest stock prices during this period.

3. **Trading Volume**:
   - Get the average trading volume for Illumina over the past month.

4. **Market Trends**:
   - Analyze any significant news or events related to Illumina during the past month that may have influenced stock performance.


In [9]:
from IPython.display import display, Markdown

In [10]:
display(Markdown(chat_history.chat_history[-1]['content']))

```md
# Illumina's Stock Price Performance: A Month in Review

**Date:** November 2, 2024

In the past month, Illumina Inc. (ticker symbol: ILMN) experienced notable fluctuations in its stock price. Here, we take a closer look at how the stock performed over this period and what factors may have influenced its movements.

## Price Overview

- **Starting Price (2024-10-02):** $136.36
- **Ending Price (2024-11-02):** $149.76
- **Percentage Change:** +9.83%

The stock opened at $136.36 and climbed significantly, reaching a peak closing price of $149.76. This represents an increase of about 9.83% over the course of the month, highlighting strong market interest and possibly positive investor sentiment.

## Highs and Lows

During October, Illumina's stock price fluctuated between a low of $136.36 and a high of $149.76. This range indicates a robust performance, as the stock not only recovered from its initial price but also achieved new heights.

## Trading Volume

The average trading volume for Illumina during this month was approximately 1,695,986 shares. A healthy trading volume often reflects investor activity and can be a good indicator of market sentiment toward the stock.

## Market Trends and Influences

While specific events weren’t detailed in our data collection, it is important to note that Illumina operates in the biotechnology sector, which can be heavily influenced by developments in research, FDA approvals, and changes in healthcare policy. Any announcements related to new product developments, partnerships, or shifts in competition could have played a significant role in the stock's performance this month.

## Comparative Analysis

To provide a broader context, it would be beneficial to compare Illumina's performance with that of its industry peers or relevant indices. This could help investors assess whether Illumina's growth is reflective of broader market trends or unique to the company.

## Buy/Sell/Wait Recommendation

Given the strong upward trajectory of Illumina's stock and the overall positive market sentiment, we suggest the following:

- **Buy:** If you are looking for long-term growth opportunities and believe in Illumina's potential within the biotechnology sector, this may be a good time to buy, especially if the fundamentals remain solid and you have confidence in upcoming developments.
  
- **Wait:** If you're uncertain about market volatility or prefer to see how the stock reacts to upcoming news or quarterly earnings reports, holding off on a purchase could be wise. This will allow you to assess whether the momentum can be sustained.

- **Sell:** If you currently own shares and are concerned about future growth prospects or see the stock nearing your target profit levels, this could be an opportune moment to take profits and re-evaluate your investment strategy in light of market developments.

## Conclusion

Overall, Illumina has shown a strong upward trend in the past month. With a solid percentage increase and active trading, it appears that investor confidence remains high. Keeping an eye on upcoming news and market conditions will be essential in understanding future movements.

Stay tuned for more updates on Illumina and other stocks in the biotechnology sector as we monitor the ever-evolving market landscape.
```

In [13]:
agentops.end_session("Success")

🖇 AgentOps: Session Stats - Duration: 3m 32.0s | Cost: $0.004431 | LLMs: 20 | Tools: 0 | Actions: 107 | Errors: 0
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=573a99df-31dd-4ff2-9a96-bd8bfbb29b7e
